In [1]:
import scanpy as sc
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.optim import Adam
import torch.utils.data as Data
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from collections import Counter

In [103]:
adata = sc.read_csv("C:\\Users\Lenovo\\Documents\\Python Scripts\\Intra-dataset\\Pancreatic_data\\Muraro\\Filtered_Muraro_HumanPancreas_data.csv")

In [104]:
label = pd.read_csv('C:\\Users\Lenovo\\Documents\\Python Scripts\\Intra-dataset\\Pancreatic_data\\Muraro\\Labels.csv')

In [57]:
label = pd.read_csv('\\Users\Lenovo\\Documents\\Python Scripts\\Intra-dataset\\Pancreatic_data\\Baron Mouse\\Labels.csv')
adata = sc.read_csv("C:\\Users\Lenovo\\Documents\\Python Scripts\\Intra-dataset\\Pancreatic_data\\Baron Mouse\\Filtered_MousePancreas_data.csv")

In [105]:
adata

AnnData object with n_obs × n_vars = 2122 × 18915

In [106]:
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)

In [107]:
label.columns = ['Label']
label.Label.value_counts()
status_dict = label['Label'].unique().tolist()
label['transfromed']=label['Label'].apply(lambda x : status_dict.index(x))
label_arr = label['transfromed'].values

In [108]:
data = adata.to_df()

In [109]:
data_arr = data.values

In [110]:
label_arr

array([0, 1, 2, ..., 0, 4, 7], dtype=int64)

In [111]:
Counter(label_arr)

Counter({0: 812, 1: 21, 2: 193, 3: 448, 4: 245, 5: 219, 6: 101, 7: 80, 8: 3})

In [112]:
X_train, X_test, y_train, y_test = train_test_split(data_arr, label_arr)

In [113]:
Counter(y_test)

Counter({0: 214, 4: 51, 3: 103, 5: 56, 7: 20, 1: 2, 2: 54, 6: 30, 8: 1})

In [114]:
dtype = torch.float

X_train=torch.from_numpy(X_train)
X_test=torch.from_numpy(X_test)
y_train = torch.from_numpy(y_train)

In [115]:
train_data= Data.TensorDataset(X_train,y_train)

In [116]:
import torch
import torch.utils.data as Data

def Weighted_Sampling(train_label = None):

    '''

    :param train_label: Input train label (in tensor) to calculate sampling weight
    :return: Pytorch Weighted Sampler
    '''

    class_sample_count = torch.tensor(
        [(train_label == t).sum() for t in torch.unique(train_label, sorted=True)])

    weight = 1. / class_sample_count.float()
    samples_weight = torch.tensor([weight[t] for t in train_label])

    sampler = Data.WeightedRandomSampler(samples_weight, len(samples_weight))
    
    return samples_weight, sampler

In [117]:
weight, sampler = Weighted_Sampling(y_train)

In [118]:
weight

tensor([0.0029, 0.0072, 0.0017,  ..., 0.0017, 0.0029, 0.0029])

In [119]:
train_loader= Data.DataLoader(dataset=train_data,batch_size=128,sampler = sampler,num_workers=1)

In [120]:
from scBalance_classifier import classifier

In [194]:
input_size = X_train.shape[1]
num_class = len(status_dict)
num_epochs = 20

model = classifier(input_size, num_class)

In [195]:
print(model)

classifier(
  (layer1): Linear(in_features=18915, out_features=128, bias=True)
  (bn1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (elu1): ELU(alpha=1.0)
  (dropout1): Dropout(p=0.5, inplace=False)
  (layer2): Linear(in_features=128, out_features=64, bias=True)
  (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (elu2): ELU(alpha=1.0)
  (dropout2): Dropout(p=0.5, inplace=False)
  (layer3): Linear(in_features=64, out_features=32, bias=True)
  (bn3): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (elu3): ELU(alpha=1.0)
  (dropout3): Dropout(p=0.5, inplace=False)
  (layer4): Linear(in_features=32, out_features=9, bias=True)
)


In [196]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
total_step = len(train_loader)

In [193]:
import torch
import torch.nn as nn

class classifier(nn.Module):
    '''
    Deep Neural Network with 3 hidden layers,
    each layer has batch normalization function locate before the activation function (ELU),
    the followed by the dropout function to reduce overfiting the reference and overcome batch effect
    '''
    def __init__(self, input_size, class_num):
        '''

        :param input_size: Input layer unit number (feature number)
        :param class_num: Number of different cell types.
        '''
        if input_size == None or class_num == None:
            raise ValueError("Must declare number of features and number of classes")
        super(classifier, self).__init__()
        self.layer1 = nn.Linear(input_size, 128)
        self.bn1 = nn.BatchNorm1d(128)
        self.elu1 = nn.ELU()
        self.dropout1 = nn.Dropout(0.5)
        self.layer2 = nn.Linear(128, 64)
        self.bn2 = nn.BatchNorm1d(64)
        self.elu2 = nn.ELU()
        self.dropout2 = nn.Dropout(0.5)
        self.layer3 = nn.Linear(64, 32)
        self.bn3 = nn.BatchNorm1d(32)
        self.elu3 = nn.ELU()
        self.dropout3 = nn.Dropout(0.5)
        self.layer4 = nn.Linear(32, class_num)


    def forward(self, x):
        '''

        :param x: forward calculation
        :return:
        '''
        out = self.layer1(x)
        out = self.bn1(out)
        out = self.elu1(out)
        out = self.dropout1(out)
        out = self.layer2(out)
        out = self.bn2(out)
        out = self.elu2(out)
        out = self.dropout2(out)
        out = self.layer3(out)
        out = self.bn3(out)
        out = self.elu3(out)
        out = self.dropout3(out)
        out = self.layer4(out)

        return out

In [197]:
import time
print("--------Start annotating----------")
start = time.perf_counter()

#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = 'cpu'
print("Computational unit be used is:", device)

model.to(device)
#loss_weight = loss_weight.to(device)

model.train()
for epoch in range(num_epochs):
    for step,(batch_x,batch_y) in enumerate(train_loader):
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)
        outputs = model(batch_x)
        train_loss = criterion(outputs, batch_y)
        
        # Backward and optimize
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()
        
    a = "="*(epoch+1)
    b = "."*(num_epochs-epoch-1)
    c = ((epoch+1)/num_epochs)*100
    dur = time.perf_counter() - start
    print("\r{:^3.0f}%[{}->{}]{:.2f}s".format(c,a,b,dur),end = '')
    time.sleep(0.1)
    
print("\n--------Annotation Finished----------")

--------Start annotating----------
Computational unit be used is: cpu
100%[====================->]37.11s
--------Annotation Finished----------


In [198]:
model.eval()
with torch.no_grad():
    X_test = X_test.to(device)
    output = model(X_test)

In [199]:
#softmax
import torch.nn.functional as F
b = F.softmax(output,dim=1)
pred = torch.argmax(b, dim = 1).cpu()

In [200]:
result = list(pred.numpy())

In [201]:
results = []
for i in result:
    results.append(status_dict[i])

In [202]:
pred

tensor([0, 0, 4, 3, 0, 0, 5, 0, 7, 0, 3, 0, 5, 0, 1, 0, 7, 3, 3, 0, 2, 5, 3, 2,
        7, 7, 0, 0, 4, 0, 4, 0, 2, 0, 0, 5, 4, 2, 0, 5, 0, 5, 6, 0, 0, 0, 4, 3,
        0, 3, 0, 6, 5, 0, 2, 0, 3, 2, 0, 2, 5, 3, 0, 0, 0, 2, 2, 3, 0, 0, 0, 3,
        3, 0, 3, 5, 0, 5, 5, 6, 0, 0, 0, 6, 2, 0, 2, 4, 7, 3, 4, 6, 2, 0, 3, 2,
        4, 0, 5, 0, 0, 0, 0, 3, 5, 3, 0, 6, 6, 3, 2, 5, 0, 2, 3, 0, 4, 3, 0, 2,
        0, 4, 0, 3, 4, 5, 7, 0, 4, 3, 0, 0, 5, 0, 5, 0, 0, 0, 0, 3, 0, 3, 3, 2,
        3, 0, 3, 0, 6, 5, 0, 7, 0, 2, 3, 5, 7, 5, 0, 3, 0, 2, 2, 0, 0, 0, 5, 7,
        6, 0, 0, 5, 0, 3, 0, 6, 0, 0, 0, 5, 6, 4, 0, 2, 3, 0, 2, 5, 2, 3, 5, 2,
        0, 0, 0, 0, 0, 0, 3, 0, 3, 2, 0, 5, 0, 0, 0, 4, 0, 0, 0, 4, 0, 0, 2, 5,
        0, 5, 0, 0, 7, 2, 0, 6, 3, 0, 3, 5, 3, 3, 0, 0, 0, 6, 5, 2, 0, 4, 3, 0,
        2, 3, 5, 7, 0, 0, 3, 3, 2, 6, 3, 0, 0, 5, 2, 3, 0, 6, 4, 2, 0, 3, 3, 3,
        6, 0, 0, 3, 0, 3, 6, 6, 0, 3, 7, 6, 4, 0, 4, 5, 3, 0, 4, 0, 0, 3, 0, 3,
        4, 0, 3, 3, 5, 0, 3, 0, 7, 0, 5,

In [203]:
y_test

array([0, 0, 4, 3, 0, 3, 5, 0, 7, 0, 3, 0, 5, 0, 1, 0, 7, 3, 3, 0, 2, 5,
       3, 2, 7, 7, 0, 0, 4, 0, 4, 0, 2, 0, 0, 5, 4, 2, 0, 5, 0, 5, 6, 0,
       0, 0, 4, 3, 0, 3, 0, 6, 5, 0, 2, 0, 3, 2, 0, 2, 5, 3, 0, 0, 0, 2,
       2, 3, 0, 0, 0, 3, 3, 0, 3, 5, 0, 5, 5, 6, 0, 0, 0, 6, 2, 0, 2, 4,
       7, 3, 4, 6, 2, 0, 3, 2, 4, 0, 5, 0, 0, 0, 0, 3, 5, 3, 0, 6, 6, 3,
       2, 5, 0, 2, 3, 0, 4, 3, 0, 2, 0, 4, 0, 3, 4, 5, 7, 0, 4, 3, 0, 0,
       5, 0, 5, 0, 0, 0, 0, 3, 0, 3, 3, 2, 3, 0, 3, 0, 6, 5, 0, 7, 0, 2,
       3, 5, 7, 5, 0, 3, 0, 2, 2, 0, 0, 0, 3, 7, 6, 0, 0, 5, 0, 3, 0, 6,
       0, 0, 0, 5, 6, 3, 0, 2, 3, 0, 2, 5, 2, 3, 4, 2, 0, 0, 0, 0, 0, 0,
       0, 0, 3, 2, 0, 5, 0, 0, 0, 4, 0, 0, 0, 4, 0, 0, 2, 5, 0, 5, 0, 0,
       7, 2, 0, 6, 3, 0, 3, 5, 3, 3, 0, 0, 0, 6, 5, 2, 0, 4, 3, 0, 2, 3,
       5, 7, 0, 0, 3, 3, 2, 6, 3, 0, 0, 5, 2, 3, 0, 6, 4, 2, 0, 3, 3, 3,
       6, 0, 0, 3, 0, 3, 6, 6, 0, 3, 7, 6, 4, 0, 4, 5, 3, 0, 4, 0, 0, 3,
       0, 3, 4, 0, 3, 3, 5, 0, 3, 0, 7, 0, 5, 3, 5,

In [204]:
f1_score(y_test,pred.cpu(),average='macro')

0.8777727728591119

In [205]:
from sklearn.metrics import confusion_matrix,cohen_kappa_score
confusion_matrix(y_test, pred)

array([[213,   0,   0,   1,   0,   0,   0,   0,   0],
       [  0,   2,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,  54,   0,   0,   0,   0,   0,   0],
       [  1,   0,   0,  99,   1,   1,   1,   0,   0],
       [  0,   0,   0,   1,  49,   1,   0,   0,   0],
       [  0,   0,   0,   0,   0,  56,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,  30,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,  20,   0],
       [  1,   0,   0,   0,   0,   0,   0,   0,   0]], dtype=int64)

In [206]:
f1 = f1_score(y_test,pred,average=None)
np.median(f1)

0.9836065573770492

In [207]:
cohen_kappa_score(y_test,pred)

0.9802877030162414